In [1]:
import os 
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\bvssv\\My_projects\\MLOps\\EndtoEndDSProject'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    


In [4]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion(self) -> DataIngestionconfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config=DataIngestionconfig(
        root_dir=config.root_dir,
        source_URL=config.source_URL,
        local_data_file=config.local_data_file,
        unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
    
    

In [6]:
import urllib.request as request
from src.datascience import logger
import zipfile

In [7]:
#component -Data ingestion

class DataIngestion:
    def __init__(self, config: DataIngestionconfig):
        self.config=config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url =self.config.source_URL,
                filename= self.config.local_data_file
            )
            
            logger.info(f"{filename} download! with following info: \n{headers}")
        
        else:
            logger.info(f"file already exists")
            
    def extract_zipfile(self):
        unzip_path =self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            
          
            
    

In [8]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion()
    data_ingestion =DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
    
except Exception as e:
    raise e
    
    

[2024-12-23 19:10:57,117 : INFO : INFO : common : yaml file : config\config.yaml loaded successfully]
[2024-12-23 19:10:57,119 : INFO : INFO : common : yaml file : params.yaml loaded successfully]
[2024-12-23 19:10:57,121 : INFO : INFO : common : yaml file : schema.yaml loaded successfully]
[2024-12-23 19:10:57,122 : INFO : INFO : common : create directory at: artifacts]
[2024-12-23 19:10:57,123 : INFO : INFO : common : create directory at: artifacts/data_ingestion]
[2024-12-23 19:10:57,124 : INFO : INFO : 2880785490 : file already exists]
